In [ ]:
import pickle
import os
from operator import itemgetter

import matplotlib
import numpy as np
import pandas as pd
import seaborn as sns
from astropy.stats import sigma_clip
from matplotlib import pyplot as plt
from matplotlib import ticker
from matplotlib.colors import LinearSegmentedColormap
from scipy.signal import savgol_filter
from statsmodels.nonparametric.kernel_regression import KernelReg

from hera_cal.io import HERAData

from simpleredcal.fit_diagnostics import abs_residuals
from simpleredcal.plot_utils import arr_pcmesh, clipped_heatmap, flagged_hist, \
plot_res_heatmap
from simpleredcal.red_likelihood import fltBad, get_reds, group_data, makeCArray
from simpleredcal.red_utils import calfits_to_flags, find_deg_df, find_flag_file, \
find_nearest, find_rel_df, find_zen_file, get_bad_ants, JD2LSTPATH, match_lst, RESPATH

In [ ]:
plt.rcParams['figure.figsize'] = (12, 8)
%matplotlib inline

In [ ]:
import matplotlib as mpl

plot_figs = False
if plot_figs:
    mpl.rcParams['figure.dpi'] = 300

mpl.rc('font',**{'family':'serif','serif':['cm']})
mpl.rc('text', usetex=True)
mpl.rc('text.latex', preamble=r'\usepackage{amssymb} \usepackage{amsmath}')

In [ ]:
JD1 = 2458098.43869
JD_comp = 2458099

pol = 'ee'
n_dist = 'gaussian'

## Getting final flags from smooth_abs calfits

In [ ]:
JD2 = match_lst(JD1, JD_comp, tint=0)
JD3 = match_lst(JD1, JD_comp, tint=-1)

In [ ]:
band_edge = 50 # frequency channels to cut
add_ba = 14 # additional bad antennas
cal_file = 'smooth_abs'

flags1 = np.mean(calfits_to_flags(JD1, cal_file, pol=pol, add_bad_ants=add_ba), \
                 axis=2).astype(int)[band_edge:-band_edge, :]
flags2 = np.mean(calfits_to_flags(JD2, cal_file, pol=pol, add_bad_ants=add_ba), \
                 axis=2).astype(int)[band_edge:-band_edge, :]
flags3 = np.mean(calfits_to_flags(JD3, cal_file, pol=pol, add_bad_ants=add_ba), \
                 axis=2).astype(int)[band_edge:-band_edge, :]

In [ ]:
# aligning flags on JD_comp wit those on JD1 in LAST

last_df = pd.read_pickle(JD2LSTPATH)
last1 = last_df[last_df['JD_time'] == JD1]['LASTs'].values[0]
last2 = last_df[last_df['JD_time'] == JD2]['LASTs'].values[0]
_, offset = find_nearest(last2, last1[0])

flagsc = np.concatenate((flags2[:, offset:], flags3[:, :offset]), axis=1) * 2

In [ ]:
flagsf = flags1 + flagsc

In [ ]:
# define colors
colors = ((1.0, 1.0, 1.0), 'C2', 'C10', (1., 0., 0.))
cmap = LinearSegmentedColormap.from_list('Custom', colors, len(colors))

fig, ax = arr_pcmesh(np.arange(1024)[band_edge:-band_edge], np.arange(60), flagsf.T, cmap=cmap, \
    xlabel='Frequency Channel', ylabel='Time Integration', clabel='Flags', \
    xlim=(0, 1023), rtn_fig_ax=True)

cbar = ax.collections[0].colorbar
cbar.set_ticks(np.array([0., 0.75, 1.5, 2.25]) + 0.375)
cbar.set_ticklabels(['False', '98', '99', 'Both'])

fig.tight_layout()
# save_fig_dir = '/Users/matyasmolnar/Desktop/Thesis/CHAP-4/FIGS'
# plt.savefig(os.path.join(save_fig_dir, '2458098_99_OCRSD_ee_flags.pdf'), bbox_inches='tight')
plt.show()

## Negative log-likelihood histograms

In [ ]:
deg_df_path = os.path.join(RESPATH, 'deg_dfs')
rel_df_path = os.path.join(RESPATH, 'rel_dfs')

In [ ]:
deg_df_j = find_deg_df(JD1, pol, 'jd', n_dist, deg_dir=deg_df_path)
df_j = pd.read_pickle(deg_df_j)
Nfreqs = df_j.index.get_level_values('freq').unique().size

In [ ]:
flagsr = flagsf.ravel(order='F').astype(int).astype(bool)
deg_values = df_j['fun'].values.astype(float)

In [ ]:
nll_upper_cut = 0.009
flagged_hist(deg_values, flagsr, xlabel=r'$-\ln(\mathcal{L}^G_\mathrm{deg})$', lower_cut=None, \
             upper_cut=nll_upper_cut, bin_width=0.0002, hist_start=0, ylim=(0, 7000))

In [ ]:
nll_suspect_slices = []
for i, n in enumerate(deg_values):
    if ~flagsr[i]:
        if n > nll_upper_cut:
            nll_suspect_slices.append(df_j.index.values[i] + (i, n))
print(*nll_suspect_slices, sep='\n')

### Median absolute normalized residual

In [ ]:
df_j[['med_abs_norm_res_Re', 'med_abs_norm_res_Im']] = df_j.apply(lambda row: \
    pd.Series(abs_residuals(row['norm_residual'])), axis=1)

df_j['med_abs_norm_res_comb'] = np.sqrt((df_j['med_abs_norm_res_Re']**2 + \
                                         df_j['med_abs_norm_res_Im']**2).values)

rman_values = df_j['med_abs_norm_res_comb'].values.astype(float)

In [ ]:
rman_upper_cut = 0.2
flagged_hist(rman_values, flagsr, xlabel=r'$\mathcal{R}_{\mathrm{man}}$', lower_cut=None, \
             upper_cut=rman_upper_cut, bin_width=0.0025, hist_start=0, ylim=(0, 3000), figsize=(6,6))

In [ ]:
rman_suspect_slices = []
for i, n in enumerate(rman_values):
    if ~flagsr[i]:
        if n > rman_upper_cut:
            rman_suspect_slices.append(df_j.index.values[i] + (i, n))
print(*rman_suspect_slices, sep='\n')

In [ ]:
plot_res_heatmap(df_j, 'fun', index='time_int1', columns='freq', clip=True, clip_pctile=95, vmin=0)

### Normalize negative log-likelihoods by visibility amplitude mean/median

In [ ]:
plot_res_heatmap(df_j, 'fun', index='time_int1', columns='freq', clip=True, clip_pctile=95)

In [ ]:
rel_df_j = pd.read_pickle(find_rel_df(JD1, pol, n_dist, rel_dir=rel_df_path))

In [ ]:
with open(os.path.join(f'{RESPATH}/rel_dfs', 'rel_df.{}.{}.md.pkl'.format(JD1, pol)), 'rb') as f:
    md = pickle.load(f)

no_unq_bls = md['no_unq_bls']
no_min_p = 5 # number of columns in df that are attributes of the SciPy OptimizeResult 
vis_df = rel_df_j.iloc[:, no_min_p:no_unq_bls*2+no_min_p]

In [ ]:
vis_df['vamp_mean'] = vis_df.apply(lambda row: \
                      np.mean(np.abs(makeCArray(row[:no_unq_bls*2].values.astype(float)))), \
                      axis=1)

vis_df['vamp_median'] = vis_df.apply(lambda row: \
                        np.median(np.abs(makeCArray(row[:no_unq_bls*2].values.astype(float)))), \
                        axis=1)

In [ ]:
rel_df_j2a = pd.read_pickle(find_rel_df(JD2, pol, n_dist, rel_dir=rel_df_path))
rel_df_j2b = pd.read_pickle(find_rel_df(JD3, pol, n_dist, rel_dir=rel_df_path))

Nfreqs = rel_df_j.index.get_level_values('freq').unique().size
Ntints = rel_df_j.index.get_level_values('time_int').unique().size

indices = ['freq', 'time_int']
rel_df_j2a = rel_df_j2a[rel_df_j2a.index.get_level_values('time_int') >= offset]
rel_df_j2a.reset_index(inplace=True)
rel_df_j2a['time_int'] = np.tile(np.arange(Ntints - offset), Nfreqs)
rel_df_j2a.set_index(indices, inplace=True)

rel_df_j2b = rel_df_j2b[rel_df_j2b.index.get_level_values('time_int') < offset]
rel_df_j2b.reset_index(inplace=True)
rel_df_j2b['time_int'] = np.tile(np.arange(Ntints - offset, Ntints), Nfreqs)
rel_df_j2b.set_index(indices, inplace=True)

rel_df_j2 = pd.concat([rel_df_j2a, rel_df_j2b])

In [ ]:
vis_df2 = rel_df_j2.iloc[:, no_min_p:no_unq_bls*2+no_min_p]

In [ ]:
vis_df2['vamp_mean'] = vis_df2.apply(lambda row: \
                       np.mean(np.abs(makeCArray(row[:no_unq_bls*2].values.astype(float)))), \
                       axis=1)

vis_df2['vamp_median'] = vis_df2.apply(lambda row: \
                         np.median(np.abs(makeCArray(row[:no_unq_bls*2].values.astype(float)))), \
                         axis=1)

In [ ]:
vis_df['vamp_mean_xdmean'] = (vis_df['vamp_mean'] + vis_df2['vamp_mean']) / 2
vis_df['vamp_median_xdmean'] = (vis_df['vamp_median'] + vis_df2['vamp_median']) / 2

In [ ]:
plot_res_heatmap(vis_df, 'vamp_mean_xdmean', clip=True, clip_pctile=97)

In [ ]:
plot_res_heatmap(vis_df, 'vamp_median_xdmean', clip=True, clip_pctile=97)

In [ ]:
vis_df.reset_index(inplace=True)
vis_df.rename(columns={'time_int': 'time_int1'}, inplace=True)
vis_df.set_index(['time_int1', 'freq'], inplace=True)

In [ ]:
df_j['mean_nrm_nll'] = df_j['fun'] / vis_df['vamp_mean']**2
df_j['median_nrm_nll'] = df_j['fun'] / vis_df['vamp_median']**2

In [ ]:
plot_res_heatmap(df_j, 'mean_nrm_nll', index='time_int1', columns='freq', clip=True, clip_pctile=97)

In [ ]:
plot_res_heatmap(df_j, 'median_nrm_nll', index='time_int1', columns='freq', clip=True, clip_pctile=97)

In [ ]:
mean_nrm_nll = df_j['mean_nrm_nll'].values.astype(float)
median_nrm_nll = df_j['median_nrm_nll'].values.astype(float)

In [ ]:
vamp_mean_nrm_nll_upper_cut = 25
flagged_hist(mean_nrm_nll, flagsr, xlabel=r'$-\ln(\mathcal{L}^G_\mathrm{deg}) \; / \; \overline{\left| V_\mathrm{obs} \right|}$', \
             lower_cut=None, upper_cut=vamp_mean_nrm_nll_upper_cut, bin_width=0.5, hist_start=0, ylim=None)

In [ ]:
vamp_med_nrm_nll_upper_cut = 8
flagged_hist(median_nrm_nll, flagsr, xlabel=r'$-\ln(\mathcal{L}^G_\mathrm{deg}) \; / \; \mathrm{med} \left( \left| V_\mathrm{obs} \right| \right)$', lower_cut=None, \
             upper_cut=vamp_med_nrm_nll_upper_cut, bin_width=0.1, hist_start=0, ylim=None)

In [ ]:
vamp_med_suspect_slices = []
for i, n in enumerate(median_nrm_nll):
    if ~flagsr[i]:
        if n > vamp_med_nrm_nll_upper_cut:
            vamp_med_suspect_slices.append(df_j.index.values[i] + (n,))
print(*vamp_med_suspect_slices, sep='\n')

### Normalizing by smoothed noise from autocorrelations

In [ ]:
# Load inferred noise on data
local_dir = '/Users/matyasmolnar/Downloads/HERA_Data/sample_data'
if os.path.exists(local_dir):
    noise_dir = local_dir
else:
    noise_dir = '/lustre/aoc/projects/hera/H1C_IDR2/IDR2_2/{}'.format(int(JD1))
noise_file = os.path.join(noise_dir, 'zen.{}.HH.noise_std.uvh5'.format(JD1))
hd_noise = HERAData(noise_file)
noise, noise_flags, _ = hd_noise.read(polarizations=[pol])
RedG = md['redg']

In [ ]:
bl_var = np.empty((RedG.shape[0], md['Ntimes'], md['Nfreqs']), dtype=complex)
for i in range(bl_var.shape[0]):
    bl_var[i, ...] = noise[(int(RedG[i, 1]), int(RedG[i, 1]), pol)] * \
                     noise[(int(RedG[i, 2]), int(RedG[i, 2]), pol)]

In [ ]:
def arr_heatmap(arr, clip_pctile=97, clip_rnd=100, sci_format=False, retn_vmax=False):
    fig, ax, vmin, vmax = clipped_heatmap(arr, 'time_int', xlabel='freq', figsize=(11,7), \
                              clip_pctile=clip_pctile, vmin=0, clip_rnd=clip_rnd, \
                              sci_format=sci_format, retn_vlims=True)
    ax.xaxis.set_major_formatter(ticker.ScalarFormatter(useOffset=-band_edge))
    plt.tight_layout()
    plt.show()
    if retn_vmax:
        return vmax

In [ ]:
arr = np.absolute(bl_var.mean(axis=0))[:, band_edge:-band_edge]
arr_heatmap(arr, clip_pctile=92)

In [ ]:
noise_nrm_nll = df_j['fun'].values.astype(float) / arr.ravel()

In [ ]:
arr = noise_nrm_nll.reshape((md['Ntimes'], Nfreqs))
vmax = arr_heatmap(arr, clip_pctile=95, clip_rnd=1e4, sci_format=True, retn_vmax=True)

In [ ]:
flagged_hist(noise_nrm_nll, flagsr, xlabel=r'$-\ln(\mathcal{L}^G_\mathrm{deg}) \; / \; \sigma_{98}^2$', \
             lower_cut=None, upper_cut=vmax, bin_width=None, hist_start=0, ylim=None)

In [ ]:
noise_nrm_98_suspect_slices = []
for i, n in enumerate(noise_nrm_nll):
    if ~flagsr[i]:
        if n > vmax:
            noise_nrm_98_suspect_slices.append(df_j.index.values[i] + (n,))
print(*noise_nrm_98_suspect_slices, sep='\n')

In [ ]:
# Load inferred noise on data from 2nd dataset
if not os.path.exists(local_dir):
    noise_dir = '/lustre/aoc/projects/hera/H1C_IDR2/IDR2_2/{}'.format(int(JD2))

noise_file = os.path.join(noise_dir, 'zen.{}.HH.noise_std.uvh5'.format(JD2))
hd_noise = HERAData(noise_file)
noise, _, _ = hd_noise.read(polarizations=[pol])

bl_var2a = np.empty((RedG.shape[0], md['Ntimes'], md['Nfreqs']), dtype=complex)
for i in range(bl_var2a.shape[0]):
    bl_var2a[i, ...] = noise[(int(RedG[i, 1]), int(RedG[i, 1]), pol)] * \
                       noise[(int(RedG[i, 2]), int(RedG[i, 2]), pol)]
    
noise_file = os.path.join(noise_dir, 'zen.{}.HH.noise_std.uvh5'.format(JD3))
hd_noise = HERAData(noise_file)
noise, _, _ = hd_noise.read(polarizations=[pol])

bl_var2b = np.empty((RedG.shape[0], md['Ntimes'], md['Nfreqs']), dtype=complex)
for i in range(bl_var2b.shape[0]):
    bl_var2b[i, ...] = noise[(int(RedG[i, 1]), int(RedG[i, 1]), pol)] * \
                       noise[(int(RedG[i, 2]), int(RedG[i, 2]), pol)]

In [ ]:
bl_var2 = np.concatenate((bl_var2a[:, offset:, :], bl_var2a[:, :offset, :]), axis=1)

In [ ]:
bl_varca = (bl_var + bl_var2) / 2

In [ ]:
hd = HERAData(find_zen_file(JD1))
reds = fltBad(get_reds(hd.antpos, pols=[pol]), get_bad_ants(find_zen_file(JD_time=JD1)))
bl_types = RedG[:, 0]
slct_bl_type_id = 2 # 14 m EW baselines
slct_bl_type = reds[slct_bl_type_id][0]
print(slct_bl_type) # 14 m EW baselines

In [ ]:
ew_14_bls = np.where(RedG[:, 0] == slct_bl_type_id)[0]

In [ ]:
bl_varcr = bl_varca[ew_14_bls, ...] # selecting only 14m EW baselines
bl_varcr = np.mean(np.abs(bl_varcr), axis=0) # average over 14m EW baselines

In [ ]:
def nan_helper(y):
    return np.isnan(y), lambda z: z.nonzero()[0]

In [ ]:
bl_varc = np.empty_like(bl_varcr)
for i in range(bl_varcr.shape[1]):
    bl_varc[:, i] = sigma_clip(bl_varcr[:, i], sigma=4, cenfunc='median').filled(fill_value=np.nan)
    nans, x = nan_helper(bl_varc[:, i])
    bl_varc[:, i][nans]= np.interp(x(nans), x(~nans), bl_varc[:, i][~nans])
    
# Savitzky-Golay filter
for i in range(bl_varc.shape[1]):
    bl_varc[:, i] = savgol_filter(bl_varc[:, i], window_length=17, polyorder=3, mode='interp')
    
bl_varc[bl_varc < 0] = 1e-8 # zero pad bottom

In [ ]:
test_freq = 381

fig, ax = plt.subplots()

ax.plot(bl_varcr[:, test_freq])
ax.plot(bl_varc[:, test_freq])

plt.tight_layout()
plt.show()

In [ ]:
test_tint = 30
clf = KernelReg(bl_varc[test_tint, :], np.arange(hd.Nfreqs), var_type='c', reg_type='ll', bw = 'cv_ls')
y_pred = clf.fit()[0]

In [ ]:
fff = np.append(np.zeros(band_edge), np.append(flagsf[:, test_tint], np.zeros(band_edge))).astype(bool)
mf = np.ma.masked_array(bl_varcr[test_tint, :], fff)

In [ ]:
fig, ax = plt.subplots()

ax.plot(bl_varcr[test_tint, :])
ax.plot(mf)
ax.plot(y_pred)

ax.set_ylim((0, 300))

plt.tight_layout()
plt.show()

In [ ]:
arr = np.absolute(bl_varca.mean(axis=0))[:, band_edge:-band_edge]
arr_heatmap(arr, clip_pctile=93)

In [ ]:
snoise_arr = np.absolute(bl_varc)[:, band_edge:-band_edge]
arr_heatmap(snoise_arr, clip_pctile=93)

In [ ]:
noise_nrm_nll = df_j['fun'].values.astype(float) / snoise_arr.ravel()

In [ ]:
arr = noise_nrm_nll.reshape((md['Ntimes'], Nfreqs))
arr_heatmap(arr, clip_pctile=95, clip_rnd=1e4, sci_format=True)

In [ ]:
noise_nrm_upper_cut = 9.8e-5
flagged_hist(noise_nrm_nll, flagsr, xlabel=r'$-\ln(\mathcal{L}^G_\mathrm{deg}) \; / \; \sigma_{14m}^2 $', \
             lower_cut=None, upper_cut=noise_nrm_upper_cut, bin_width=vmax/50, hist_start=0, logy=False, \
             ylim=None, figsize=(6, 6))

In [ ]:
noise_nrm_suspect_slices = []
for i, n in enumerate(noise_nrm_nll):
    if ~flagsr[i]:
        if n > noise_nrm_upper_cut:
            tint, freq = df_j.index.values[i]
            noise_nrm_suspect_slices.append(df_j.index.values[i] + \
                (i, n, rman_values[i], deg_values[i], snoise_arr.ravel()[i],))
            
rman_adj_slices = []
for i in rman_suspect_slices:
    i = i[:3] + (noise_nrm_nll[i[2]], i[3], deg_values[i[2]], snoise_arr.ravel()[i[2]],)
    rman_adj_slices.append(i)            
            
noise_nrm_suspect_slices += rman_adj_slices
outliers = sorted(noise_nrm_suspect_slices, key=itemgetter(1))
print(*outliers, sep='\n')

## Outlier detection

In [ ]:
vis_list = list(map(str, np.arange(md['no_unq_bls']*2).tolist()))

visC_df = vis_df[vis_list].apply(lambda row: makeCArray(row.values), axis=1)
visC_df = pd.DataFrame(visC_df.values.tolist(), index=visC_df.index)

In [ ]:
def outlier_heatmap(arr, vmin=0, vmax=None, clip_pctile=98, cmap=sns.cm.rocket_r, center=None):
    fig, ax = plt.subplots(figsize=(11, 7))
    if vmax is None:
        vmax = np.ceil(np.nanpercentile(arr, clip_pctile)*10000)/10000
    ax = sns.heatmap(arr, vmax=vmax, vmin=vmin, cmap=cmap, center=center)
    ax.xaxis.set_major_locator(ticker.MultipleLocator(50))
    ax.xaxis.set_major_formatter(ticker.ScalarFormatter(useOffset=-band_edge))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(5))
    ax.yaxis.set_major_formatter(ticker.ScalarFormatter())
    ax.set_xlabel('freq')
    ax.set_ylabel('time_int')

    # scale ellipse to appear as a circle
    circ_height = 2
    width = circ_height * Nfreqs/md['Ntimes'] * 8 / 11

    for suspect in outliers:
        circle = matplotlib.patches.Ellipse((suspect[1]+0.5, suspect[0]+0.5), width, circ_height, \
                                             color='cyan', fill=False, lw=1.3)
        ax.add_patch(circle)

    plt.tight_layout()
    plt.show()

In [ ]:
def outlier_pcmesh(arr, vmin=0, vmax=None, clip_pctile=98, cmap=sns.cm.rocket_r, \
                   clabel=None, c_extend=None, sc_color='cyan', savefig=None, dpi=125):

    if vmax is None:
        vmax = np.ceil(np.nanpercentile(arr, clip_pctile)*10000)/10000
        
    fig, ax = arr_pcmesh(np.arange(hd.Nfreqs)[band_edge:-band_edge],np.arange(hd.Ntimes), arr, cmap=cmap, \
        xlabel='Frequency Channel', ylabel='Time Integration', clabel=clabel, vmax=vmax, \
        xlim=(0, 1023), rtn_fig_ax=True)

    for suspect in outliers:
        ax.scatter(suspect[1]+band_edge, suspect[0], s=100, fc='None', edgecolors=sc_color, lw=1, ls='-')

    fig.tight_layout()
    if savefig is not None:
        plt.savefig(savefig, bbox_inches='tight')
    plt.show()

In [ ]:
visC_14ewabs = visC_df[2].abs().values.reshape((Nfreqs, md['Ntimes'])).transpose()

outlier_pcmesh(visC_14ewabs, vmin=0, vmax=0.06, clabel=r'$|V|$', c_extend='max', savefig=None)

In [ ]:
visC_14ewangle = np.angle(visC_df[2].values).reshape((Nfreqs, md['Ntimes'])).transpose()

outlier_pcmesh(visC_14ewangle, vmin=n-np.pi, vmax=np.pi, clabel=r'$\varphi$', cmap='PiYG', \
               sc_color='darkorange')

In [ ]:
rel_df_j2a = pd.read_pickle(find_rel_df(JD2, pol, n_dist, rel_dir=rel_df_path))
rel_df_j2b = pd.read_pickle(find_rel_df(JD3, pol, n_dist, rel_dir=rel_df_path))

Nfreqs = rel_df_j.index.get_level_values('freq').unique().size
Ntints = rel_df_j.index.get_level_values('time_int').unique().size

indices = ['freq', 'time_int']
rel_df_j2a = rel_df_j2a[rel_df_j2a.index.get_level_values('time_int') >= offset]
rel_df_j2a.sort_index()
rel_df_j2a.reset_index(inplace=True)
rel_df_j2a['time_int'] = np.tile(np.arange(Ntints - offset), Nfreqs)
rel_df_j2a.set_index(indices, inplace=True)

rel_df_j2b = rel_df_j2b[rel_df_j2b.index.get_level_values('time_int') < offset]
rel_df_j2b.sort_index()
rel_df_j2b.reset_index(inplace=True)
rel_df_j2b['time_int'] = np.tile(np.arange(Ntints - offset, Ntints), Nfreqs)
rel_df_j2b.set_index(indices, inplace=True)

# rel_df_j2 = pd.concat([rel_df_j2a, rel_df_j2b])
rel_df_j2 = rel_df_j2a.append(rel_df_j2b)
rel_df_j2.sort_index()

vis_df2 = rel_df_j2a.iloc[:, no_min_p:no_unq_bls*2+no_min_p]

In [ ]:
visC_df2a = vis_df2[vis_list].apply(lambda row: makeCArray(row.values), axis=1)
visC_df2a = pd.DataFrame(visC_df2a.values.tolist(), index=visC_df2a.index)

In [ ]:
vis_df2b = rel_df_j2b.iloc[:, no_min_p:no_unq_bls*2+no_min_p]
visC_df2b = vis_df2b[vis_list].apply(lambda row: makeCArray(row.values), axis=1)
visC_df2b = pd.DataFrame(visC_df2b.values.tolist(), index=visC_df2b.index)

In [ ]:
visC_14ewabs2a = visC_df2a[2].abs().values.reshape((Nfreqs, md['Ntimes'] - offset)).transpose()
visC_14ewabs2b = visC_df2b[2].abs().values.reshape((Nfreqs, offset)).transpose()
visC_14ewabs2 = np.append(visC_14ewabs2a, visC_14ewabs2b, axis=0)

outlier_pcmesh(visC_14ewabs2, vmin=0, vmax=0.06, clabel=r'$|V|$', c_extend='max')

In [ ]:
visC_14ewabs2a = np.angle(visC_df2a[2].values).reshape((Nfreqs, md['Ntimes'] - offset)).transpose()
visC_14ewabs2b = np.angle(visC_df2b[2].values).reshape((Nfreqs, offset)).transpose()
visC_14ewangle2 = np.append(visC_14ewabs2a, visC_14ewabs2b, axis=0)

outlier_pcmesh(visC_14ewangle2, vmin=n-np.pi, vmax=np.pi, clabel=r'$\varphi$', cmap='PiYG', \
               sc_color='darkorange')

In [ ]:
zen_fn = find_zen_file(JD1)
bad_ants = get_bad_ants(zen_fn)
flags_fn = find_flag_file(JD1, 'first')

hdraw, RedG, cMData = group_data(zen_fn, pol, chans=None, tints=None, \
                                 bad_ants=bad_ants, flag_path=flags_fn)
cData_1 = cMData.filled()
cData_1 = cData_1[band_edge:-band_edge, :, ew_14_bls[0]]

In [ ]:
raw_abs_12_13 = np.abs(cData_1).transpose()

outlier_pcmesh(raw_abs_12_13, vmin=0, vmax=0.08, clabel=r'$|V|$', c_extend='max')

In [ ]:
raw_phase_12_13 = np.angle(cData_1).transpose()

outlier_pcmesh(raw_phase_12_13, vmin=n-np.pi, vmax=np.pi, clabel=r'$\varphi$', cmap='PiYG', \
               sc_color='darkorange')

In [ ]:
zen_fn2 = find_zen_file(JD2)
bad_ants2 = get_bad_ants(zen_fn2)
flags_fn2 = find_flag_file(JD2, 'first')

hdraw, RedG, cMData = group_data(zen_fn2, pol, chans=None, tints=None, \
                                 bad_ants=bad_ants2, flag_path=flags_fn2)
cData_2 = cMData.filled()
cData_2 = cData_2[band_edge:-band_edge, :, ew_14_bls[0]]

zen_fn3 = find_zen_file(JD3)
bad_ants3 = get_bad_ants(zen_fn3)
flags_fn3 = find_flag_file(JD3, 'first')

hdraw, RedG, cMData = group_data(zen_fn3, pol, chans=None, tints=None, \
                                 bad_ants=bad_ants3, flag_path=flags_fn3)
cData_3 = cMData.filled()
cData_3 = cData_3[band_edge:-band_edge, :, ew_14_bls[0]]

cData_4 = np.concatenate((cData_2[:, offset:], cData_3[:, :offset]), axis=1)

In [ ]:
raw_abs_99_12_13 = np.abs(cData_4).transpose()

outlier_pcmesh(raw_abs_99_12_13, vmin=0, vmax=0.08, clabel=r'$|V|$', c_extend='max')

In [ ]:
raw_phase_99_12_13 = np.angle(cData_4).transpose()

outlier_pcmesh(raw_phase_99_12_13, vmin=n-np.pi, vmax=np.pi, clabel=r'$\varphi$', cmap='PiYG', \
               sc_color='darkorange')